In [7]:
#bea flowers
#COMP 498 Gen AI
#Assignment 2

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
text = """One must have a mind of winter
To regard the frost and the boughs
Of the pine-trees crusted with snow;
And have been cold a long time
To behold the junipers shagged with ice,
The spruces rough in the distant glitter
Of the January sun; and not to think
Of any misery in the sound of the wind,
In the sound of a few leaves,
Which is the sound of the land
Full of the same wind
That is blowing in the same bare place
For the listener, who listens in the snow,
And, nothing himself, beholds
Nothing that is not there and the nothing that is.
"""
#ended up being a test to see if i could extract the last words. extracts WITH punctuation...
#ultimatly not used in later code but felt like a good starting point

# Split the text into individual lines
lines = text.strip().split('\n')

last_words = []
for line in lines:
    # Split each line into words
    words = line.split()
    if words: # Check if the line is not empty
        last_word = words[-1] # Get the last word
        last_words.append(last_word)

# Print the last word of each line
for i, word in enumerate(last_words):
    print(f"Last word of line {i+1}: {word}")


Last word of line 1: winter
Last word of line 2: boughs
Last word of line 3: snow;
Last word of line 4: time
Last word of line 5: ice,
Last word of line 6: glitter
Last word of line 7: think
Last word of line 8: wind,
Last word of line 9: leaves,
Last word of line 10: land
Last word of line 11: wind
Last word of line 12: place
Last word of line 13: snow,
Last word of line 14: beholds
Last word of line 15: is.


In [5]:
from transformers import AutoModelForCausalLM

# Load the GPT-2 model
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

# Set the model to evaluation mode
model.eval()
print("GPT-2 model loaded and set to evaluation mode.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT-2 model loaded and set to evaluation mode.


In [25]:
import torch

modified_lines = []

for i, line in enumerate(lines):
    # Reconstruct the line without the last word to use as a prompt
    words = line.split()
    if words:
        # Join all words except the last one to form the prompt
        prompt = " ".join(words[:-1])

        # Encode the prompt
        input_ids = tokenizer.encode(prompt, return_tensors='pt')

        # Get model predictions
        with torch.no_grad():
            outputs = model(input_ids)
            predictions = outputs.logits

        # Get the predicted words for the token and sort them by most likely to least likely
        next_token_logits = predictions[0, -1, :]
        sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True)

        # Get the xth most likely token
        x = 14567
        # Ensure there are at least x possible tokens
        if len(sorted_indices) >= x:
            xth_most_likely_token_id = sorted_indices[(x-1)].item() #bc arrays start at 0
        else:
            # Fallback if fewer than x tokens are predicted (e.g., if prompt is empty or very short)
            xth_most_likely_token_id = sorted_indices[0].item() # Take the most likely if not enough

        # Decode the token to get the word
        predicted_word = tokenizer.decode(xth_most_likely_token_id)

        # Clean up the predicted word (remove leading/trailing spaces, handle special tokens if any)
        predicted_word = predicted_word.strip()

        # Replace the last word of the original line with the predicted word
        original_words = line.split()
        if original_words:
            new_line = " ".join(original_words[:-1] + [predicted_word])
        else:
            new_line = predicted_word # Should not happen if prompt is built as words[:-1]
    else:
        new_line = "" # Keep empty lines empty

    modified_lines.append(new_line)

# Display the modified lines
print("Modified lines:")
for ml in modified_lines:
    print(ml)

Modified lines:
One must have a mind of blogging
To regard the frost and the directors
Of the pine-trees crusted with geographical
And have been cold a long lets
To behold the junipers shagged with 480
The spruces rough in the distant apprentice
Of the January sun; and not to rem
Of any misery in the sound of the frenzy
In the sound of a few hiring
Which is the sound of the Hydro
Full of the same flew
That is blowing in the same bare talk
For the listener, who listens in the Sultan
And, nothing himself, flattering
Nothing that is not there and the nothing that undis


In [18]:
# Get the vocabulary size from the tokenizer to see what range of numbers I can use
vocabulary_size = tokenizer.vocab_size
print(f"The GPT-2 model has a vocabulary size of: {vocabulary_size} possible predictions.")

The GPT-2 model has a vocabulary size of: 50257 possible predictions.
